In [33]:
from selenium import webdriver 
from selenium.webdriver.common.by import By 
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC 
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException
import pymongo
import time
from config import EMAIL, GEN_PASS
from datetime import datetime, timedelta

In [2]:
def initialize():
    print(f"Run started at {startTime}")
#    option = webdriver.ChromeOptions()
#    option.add_argument("--incognito")
#    option.add_argument("--window-size=1440,800")
    profile = webdriver.FirefoxProfile()
    profile.set_preference("browser.privatebrowsing.autostart", True)
    browser = webdriver.Firefox(executable_path='geckodriver.exe', firefox_profile=profile)
    return browser

In [3]:
def fbLogin(driver):
    driver.get('https://www.facebook.com/')
    WebDriverWait(driver, delay).until(EC.presence_of_element_located((By.ID, "email")))
    driver.find_element_by_id('email').send_keys(EMAIL)
    passField = driver.find_element_by_id('pass')
    passField.send_keys(GEN_PASS)
    passField.send_keys(Keys.RETURN)

In [4]:
def retrievePosts():
    results = browser.find_elements_by_xpath('//div[@role="article"]')
    for res in results:
        try:
            articleLink = res.find_element_by_xpath('./div[2]/div[2]/a[1]').get_attribute('href')
            articleLinks.append(articleLink)
        except:
            continue
    return articleLinks

In [5]:
def commentScrape():
    browser.get(link)
    WebDriverWait(browser, delay).until(EC.presence_of_element_located((By.ID, "m_story_permalink_view")))
    timeout = datetime.now() + timedelta(minutes=10)
    try:
        while browser.find_element_by_id(f'see_next_{postID}') and datetime.now() < timeout:
            commentsFields()
            browser.find_element_by_id(f'see_next_{postID}').find_element_by_xpath('./a').click()
            time.sleep(1)
    except NoSuchElementException:
        try:
            commentsFields()
        except:
            pass

In [6]:
def commentsFields():
    container = browser.find_element_by_id('m_story_permalink_view')
    comments = container.find_elements_by_xpath('./div[2]/div/div[5]/div')
    for comment in comments:
        try:
            commentID = comment.get_attribute('id')
            print(commentID)
            if commentID != f'see_next_{postID}' and commentID != f'see_prev_{postID}':
                commentText = comment.find_element_by_xpath('./div/div[1]').text
                try:
                    reactionsLinkElem = comment.find_element_by_xpath('./div/div[3]/span[1]/span/a[1]')
                    if 'Like' not in reactionsLinkElem.text:
                        reactionsLink = comment.find_element_by_xpath('./div/div[3]/span[1]/span/a[1]').get_attribute('href')
                        reactions = gatherReactions(reactionsLink)
                    else:
                        reactions = []
                        reactionsLink = ''
                except:
                    reactionsLink = ''
                    reactions = []
                try:
                    authorElem = comment.find_element_by_xpath('./div/h3/a')
                except:
                    authorElem = ""
                try:
                    author = authorElem.text
                    profile = authorElem.get_attribute('href')
                except:
                    author = ''
                    profile = ''
                try:
                    repliesLink = comment.find_element_by_id(f'comment_replies_more_1:{postID}_{commentID}').find_element_by_xpath('./div[2]/a').get_attribute('href')
                    gatherReplies(repliesLink, commentID)
                except:
                    repliesLink = ''
                commentsStorage[commentID] = {
                    "article": link,
                    "articleID": postID,
                    "commentID": commentID,
                    "text": commentText,
                    "reactions": {key:int(value) for (key, value) in reactions},
                    "replies": repliesLink,
                    "reactionsLink": reactionsLink,
                    "author": author,
                    "profile": profile
                    }
                total = sum([int(value) for (key, value) in reactions])
                commentsStorage[commentID]["reactions"]["total"] = total
                search = commentsCollection.find({"commentID":commentsStorage[commentID]['commentID']})
                if len([r for r in search]) == 0:
                    commentsCollection.insert_one(commentsStorage[commentID])
                else:
                    commentsCollection.update_one({"commentID":commentsStorage[commentID]['commentID']},
                                                   {'$set': commentsStorage[commentID]})
        except StaleElementReferenceException:
            continue

In [23]:
def gatherReactions(reactionsLink, replyID=''):
    reactionsList = []
    browser3.get(reactionsLink)
    WebDriverWait(browser3, delay).until(EC.presence_of_element_located((By.CLASS_NAME, "z")))
    reactionCounts = browser3.find_element_by_class_name('z').find_elements_by_tag_name('a')
    for reaction in reactionCounts:
        reactionLink = reaction.get_attribute('href')
        if 'reaction_type' in reactionLink:
            val = reactionLink.split('reaction_type=')[1].split('&')[0]
            reactType = reactionsDict[val]
            count = reactionLink.split('total_count=')[1].split('&')[0]
            reactionsList.append((reactType, count))
    gatherReactionAuthors(browser3, replyID)
    return reactionsList

In [ ]:
# I need to figure out a unique identifier for each reaction so that it can be inserted into the reactions collection.
# I think the best way to do this would probably be create a string from the author profile, commentID, and replyID
# to use as the key, and then have the reaction type as the value. We don't need the name for this because we can put their
# name in the Profiles collection.
def gatherReactionAuthors(browser3, replyID):
    reacts = browser3.find_element_by_class_name('be').find_elements_by_tag_name("tr")
    for react in reacts:
        reactImages = react.find_elements_by_tag_name('img')
        for reactImage in reactImages:
            reactImg = reactImage.get_attribute('alt')
            if reactImg in reactionsDict:
                reactResponse = reactionsDict[reactImg]
        reactAuthorProfile = react.find_element_by_xpath('./td[3]/div/h3[1]/a').get_attribute('href')
        reactionsStorage[reactAuthorProfile][commentID][replyID]["react"] = reactResponse
        search = reactionsCollection.find({"author":reactionsStorage[reactAuthorProfile]['replyID']})
        if len([r for r in search]) == 0:
            reactionsCollection.insert_one(reactionsStorage[reactAuthorProfile])
        else:
            repliesCollection.update_one({"replyID":repliesStorage[replyID]['replyID']},
                                                   {'$set': repliesStorage[replyID]})

In [8]:
def gatherReplies(repliesLink, commentID):
    browser2.get(repliesLink)
    WebDriverWait(browser2, delay).until(EC.presence_of_element_located((By.ID, commentID)))
    try:
        while browser2.find_element_by_xpath(f'//div[@id = {commentID}]/following-sibling::div').find_element_by_xpath('./div'):
            repliesFields(commentID)
            browser2.find_element_by_xpath(f'//div[@id = {commentID}]/following-sibling::div').find_element_by_xpath('./div/a').click()
            time.sleep(1)
    except NoSuchElementException:
        try:
            print("NoSuchElementException")
            repliesFields(commentID)
        except:
            print("Failed on gathering replies")

In [39]:
def repliesFields(commentID):
    responses = browser2.find_element_by_xpath(f'//div[@id = {commentID}]/following-sibling::div').find_elements_by_xpath('./div')
    for response in responses:
        replyID = response.get_attribute('id')
        if replyID == f"comment_replies_more_2:{postID}_{commentID}":
            response.find_element_by_xpath('./a').click()
            time.sleep(1)
            repliesFields(commentID)
        if replyID != f"comment_replies_more_1:{postID}_{commentID}":
            replyText = response.find_element_by_xpath('./div/div[1]').text
            replyAuthorElem = response.find_element_by_tag_name('h3').find_element_by_tag_name('a')
            replyAuthor = replyAuthorElem.text
            replyAuthorProfile = replyAuthorElem.get_attribute('href')
            reactionsLinkElem = response.find_element_by_id(f'like_{postID}_{replyID}').find_element_by_xpath('./span/a[1]')
            if 'Like' not in reactionsLinkElem.text:
                replyReactionsLink = reactionsLinkElem.get_attribute('href')
                replyReactions = gatherReactions(replyReactionsLink, replyID)
            else:
                replyReactions = []
            repliesStorage[replyID] = {
                    "postID": postID,
                    "commentID": commentID,
                    "replyID": replyID,
                    "text": replyText,
                    "reactions": {key:int(value) for (key, value) in replyReactions},
                    "author": replyAuthor,
                    "profile": replyAuthorProfile
                    }
            search = repliesCollection.find({"replyID":repliesStorage[replyID]['replyID']})
            if len([r for r in search]) == 0:
                repliesCollection.insert_one(repliesStorage[replyID])
            else:
                repliesCollection.update_one({"replyID":repliesStorage[replyID]['replyID']},
                                                       {'$set': repliesStorage[replyID]})

In [10]:
def pullFromMongo(coll):
    search = coll.find()
    dictVar = {}
    for r in search:
        dictVar[r['commentID']] = r
    return dictVar

In [11]:
startTime = datetime.now()
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
db = client.dt_posts
commentsCollection = db.comments
repliesCollection = db.replies
reactionsCollection = db.reactions

In [12]:
commentsStorage, repliesStorage, reactionsStorage = pullFromMongo(commentsCollection), pullFromMongo(repliesCollection), pullFromMongo(reactionsCollection)
delay = 10
articleLinks, articles = [], []
reactionsDict = {
        "all": "total",
        "1": "like",
        "2": "love",
        "3": "wow",
        "4": "haha",
        "7": "sad",
        "8": "angry",
        "Like": "like",
        "Love": "love",
        "Haha": "haha",
        "Angry": "angry",
        "Wow": "wow",
        "Sad": "sad"
        }

In [22]:
browser = initialize()
browser2 = initialize()
browser3 = initialize()
fbLogin(browser)
fbLogin(browser2)
fbLogin(browser3)

Run started at 2018-12-15 21:14:16.883627
Run started at 2018-12-15 21:14:16.883627
Run started at 2018-12-15 21:14:16.883627


In [24]:
browser.get('https://mobile.facebook.com/pg/DonaldTrump/posts/')
WebDriverWait(browser, delay).until(EC.presence_of_element_located((By.ID, "structured_composer_async_container")))
for i in range(5):
    articles.extend(retrievePosts())
    browser.find_element_by_id('structured_composer_async_container').find_element_by_xpath('./div[2]/a').click()
    time.sleep(2)

In [40]:
for link in articleLinks:
    postID = link.split('story_fbid=')[1].split('&')[0]
    commentScrape()
print(f"Run completed at {datetime.now()}")
print(f"Elapsed time: {datetime.now() - startTime}")

10161899713650725
10161899713600725
10161899711650725
10161899701540725
10161899711725725
10161899709215725
10161899717370725
10161899713395725
10161899710600725
10161899759025725
see_next_10161899619985725
10161899299460725
10161899310015725
10161899297630725
10161899300555725
10161899306340725
10161899331890725
10161899301415725
10161899322365725
275514776470780
10161899329470725
see_next_10161899293050725
328191637769299
296978427595154
10161896415925725
10161896428185725
296986850927645
10161896422530725
10161896421660725
10161896421035725
10161896430090725
350024219063067
see_next_10161896407450725
see_prev_10161896407450725
328192771102519
130413477959503
10161896428455725
10161896443850725
10161896439585725
10161896446615725
10161896413970725
10161896417255725
10161896432480725
10161896420545725
see_next_10161896407450725
see_prev_10161896407450725
328191634435966
10161896413865725
10161896425895725
328197874435342
10161896412365725
10161896457740725
10161896460900725
1016189645

208931200030895
2099893850251258
2062603207319643
280934969436392
2375135015849799
2175477709368111
2371145309626462
206844786865929
see_next_10161896041690725
see_prev_10161896041690725
292431721405658
779556132385194
2201922733415998
454321098428382
191335321821417
10161896255905725
10161896198280725
506521079842298
570189353404676
372868753273056
see_next_10161896041690725
see_prev_10161896041690725
463645147496673
197670737838798
295138654459339
2371062949634698
463635847497603
2177800502437911
362638771165000
296982007825861
352361012231614
264508260886889
see_next_10161896041690725
see_prev_10161896041690725
717531225286981
914676118921338
10161896186275725
265427607469727
618068978596455
2279321455419685
717522558621181
2199323383711476
528077901028540
593888124384240
see_next_10161896041690725
see_prev_10161896041690725
299358287351059
10161896140075725
2177882009096427
463627224165132
2709399185951928
206460740261241
10161896284900725
2221975208081940
1859397657502392
10161896

10161896077410725
10161896289770725
10161896098005725
360688814490162
293595297800901
2709399742618539
see_next_10161896041690725
see_prev_10161896041690725
568606633567693
2168467066703749
135972927396777
10161896062335725
10161896192290725
1208620219292283
265428877469600
463649000829621
10161896058335725
10161896066235725
see_next_10161896041690725
see_prev_10161896041690725
10161896103700725
373207480120950
199783164290120
10161896129665725
130736117929014
2189053801310745
206460340261281
369630383803240
2189050461311079
1154751644687121
see_next_10161896041690725
see_prev_10161896041690725
10161896096135725
1181522021996116
463631724164682
463631274164727
10161896080540725
463629610831560
197658431173362
292432481405582
197678514504687
2709398739285306
see_next_10161896041690725
see_prev_10161896041690725
152373679065855
463633604164494
206463570260958
357395968142340
226952408202638
10161896057555725
10161896056030725
197675284505010
547324395739442
938649479856243
see_next_10161

157403285226156
300912133881255
10161895815795725
10161895795560725
275631149802923
10161895746910725
218779032356047
see_next_10161895735245725
see_prev_10161895735245725
10161895779670725
10161895739975725
301002877423634
2008145596144707
10161895825160725
10161895766955725
589284711530735
589283538197519
589280838197789
2008142856144981
see_next_10161895735245725
see_prev_10161895735245725
10161895756115725
2008146129477987
10161895765135725
589279254864614
712618335789592
205455810385684
157409135225571
10161895764655725
10161895773170725
10161895825510725
see_next_10161895735245725
see_prev_10161895735245725
589283634864176
10161895803505725
10161895769510725
300916493880819
157403361892815
10161895741465725
347789529344044
157407848559033
589290691530137
765068320499921
see_next_10161895735245725
see_prev_10161895735245725
347790976010566
10161895760880725
10161895795445725
10161895755665725
297557630868067
10161895823155725
275643626468342
205455827052349
10161895789570725
15740

see_next_10161895735245725
see_prev_10161895735245725
747266402275564
10161895821340725
1958397177795014
550779332001250
765065240500229
10161895806935725
325543134953080
935262593330899
134227850903166
319787735284748
see_next_10161895735245725
see_prev_10161895735245725
275645193134852
589282438197629
2148815962096997
1084239938435354
128248398181848
367013663859063
355097541970603
300815240564495
188887848734402
128241271515894
see_next_10161895735245725
see_prev_10161895735245725
276921953011946
747267865608751
10161895804540725
820561228275071
778965809113471
322406568376138
1808528549256005
765066393833447
706920116347035
300678407232557
see_next_10161895735245725
see_prev_10161895735245725
589286774863862
2138244446416971
325093104992687
2368325229847541
1350983438377715
973470516185209
314141639203487
273212410035819
275644406468264
323199781612790
see_next_10161895735245725
see_prev_10161895735245725
589282461530960
10161895813295725
1066044463575717
157407001892451
1074443346

...

10161871678790725
983556561844574
570609616693286
368874453924617
2246276828942519
344217079727219
368874200591309
265743737438666
368873653924697
see_next_10161871342075725
see_prev_10161871342075725
10161871646460725
738255716545379
368875013924561
270541543655360
738252713212346
746427649038899
10161871730990725
344220943060166
738259449878339
2246278042275731
see_next_10161871342075725
see_prev_10161871342075725
365540937531338
10161871716970725
10161871714080725
344216969727230
268932167138444
2246278718942330
2249106881768649
326027014794477
576779159448467
738261146544836
see_next_10161871342075725
see_prev_10161871342075725
2152891901435719
2516943658320623
352148732030237
10161871719670725
195449984728595
670570496677562
2092622310773873
322605788332055
263939414247473
10161871722670725
see_next_10161871342075725
see_prev_10161871342075725
2284003658276374
517438559077951
10161871769680725
368884260590303
10161871667535725
576780076115042
272483446744439
368876360591093
368876

344220099726917
272643920114233
511900569329698
326022938128218
see_next_10161871342075725
see_prev_10161871342075725
2025688761062789
491889184637029
325984301465415
2092748484094589
10161871752915725
361280344434215
274775733182540
123208872029634
506901296496581
295793301040862
see_next_10161871342075725
see_prev_10161871342075725
125714628436370
371340636948152
983558208511076
368874830591246
2434498390145637
318323122107488
275102266483154
479572915783159
2102241093155755
564941223965201
see_next_10161871342075725
see_prev_10161871342075725
339749409937226
328831574391942
2299753780300247
129450528061954
352146722030438
593562904414019
565287933934670
368874700591259
2128892347425123
572589669833876
see_next_10161871342075725
see_prev_10161871342075725
533663410482966
201331197416942
2152893731435536
368882770590452
272486460077471
2144015919183602
2244987882381579
262735911065557
2232699087011675
389732991765305
see_next_10161871342075725
see_prev_10161871342075725
38582077529396

10161871175995725
see_next_10161871146645725
see_prev_10161871146645725
10161871342200725
777903029219390
10161871179440725
2006136209679852
296258007678084
10161871257975725
10161871312720725
2006145776345562
10161871236085725
10161871293620725
see_next_10161871146645725
see_prev_10161871146645725
10161871237230725
2006128763013930
1184588971699718
10161871201795725
10161871271775725
10161871193035725
10161871181815725
10161871214830725
777908799218813
10161871307080725
see_next_10161871146645725
see_prev_10161871146645725
10161871212715725
402695626938701
777909452552081
1004011843116230
777895869220106
791887421147900
777890589220634
777898292553197
10161871317640725
10161871201585725
see_next_10161871146645725
see_prev_10161871146645725
10161871257775725
10161871171675725
777900502552976
303457293608862
312643462676888
2006130089680464
10161871362015725
1596991857067787
2006120703014736
402693556938908
see_next_10161871146645725
see_prev_10161871146645725
777893755886984
7779121325

see_prev_10161871146645725
316872282373531
138122340507095
132032121127052
1065350666976189
2138440116407735
364910434267127
402693490272248
2260986784158362
2006127103014096
2346273898936257
see_next_10161871146645725
see_prev_10161871146645725
349958942483374
309311033126554
2231793537144812
697998020593686
1004027859781295
567539103671033
364648097447061
10161871163450725
2095948650718484
115511282813761
see_next_10161871146645725
see_prev_10161871146645725
535738303561721
10161871349910725
1917180391710688
388106231932090
118701855819339
2006164349677038
2006164156343724
343801719777320
2006150716345068
10161871309415725
see_next_10161871146645725
see_prev_10161871146645725
391015864980038
362221280992548
1915500968525865
2006124426347697
268675603808003
2305701556328026
664408290623132
115052799526475
624063021325331
427838857752176
see_next_10161871146645725
see_prev_10161871146645725
390339861708861
132037491126515
365820587312156
568388996941813
2232013550166377
106535677364224

2644273842280197
390244365051744
777888822554144
2248930642033274
1004027396448008
10161871156380725
2260982624158778
364647577447113
see_next_10161871146645725
see_prev_10161871146645725
10161871276895725
199488894292748
777904942552532
326855694566732
728065350908933
103618854015617
2301250566615936
273121620058791
263695497648542
10161871364350725
see_next_10161871146645725
see_prev_10161871146645725
777926442550382
138118123840850
10161871223555725
464671010728134
319188528919247
343801606443998
341298296423716
346852682766191
520421858437874
209308113313580
see_next_10161871146645725
see_prev_10161871146645725
548297285581628
2006154696344670
10161871302210725
350520095509675
2305147256380981
273917886814622
278177849718208
10161871172185725
10161871168690725
227306971500999
see_next_10161871146645725
see_prev_10161871146645725
319191582252275
777916969217996
557211658061456
1676921885942636
1004020776448670
10161871157255725
1004016149782466
2006126683014138
312268726289348
20803

25